# Tutorial: From a CNN model to a DYNAP-CNN DevKit

This tutorial explains all steps necessary to convert a torch CNN model to a configuration of the DYNAP-CNN chip. We will first convert the network to a spiking neural network (SNN) and then to a `DynapcnnCompatibleNetwork` – a model that is compatible with the chip and simulates its behavior. Finally we port the model to a DYNAP-CNN DevKit.

## Import libraries

Before we start, we will import the libraries necessary to define a torch model, convert it to an SNN and to convert the SNN to a `DynapcnnCompatibleNetwork`.

In [1]:
%%capture

# Suppress warnings (This is only to keep the notebook pretty. You might want to comment the below two lines)
import warnings

warnings.filterwarnings("ignore")

# - Import statements
import torch
import samna
from tqdm.auto import tqdm
import numpy as np
import torch.nn as nn
from torchvision import datasets
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import io
from sinabs.backend.dynapcnn import DynapcnnCompatibleNetwork

## CNN definition

First we will define a sequential CNN model.

*Note that although non-sequential models are supported by the hardware, this is not yet the case for this library.*

In [2]:
# - Define CNN model

ann = nn.Sequential(
    nn.Conv2d(1, 20, 5, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Conv2d(20, 32, 5, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Conv2d(32, 128, 3, 1, bias=False),
    nn.ReLU(),
    nn.AvgPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(128, 500, bias=False),
    nn.ReLU(),
    nn.Linear(500, 10, bias=False),
)

# Load pre-trained weights
ann.load_state_dict(torch.load("../../../examples/mnist_params.pt", map_location="cpu"))

<All keys matched successfully>

Note that there are 5 parameter layers in the above defined model. You will see this come into play later.

## Convert to Spiking CNN

We can use the `from_torch` method from SINABS to convert our CNN to a SNN. The returned object contains the original CNN as `analog_model` and the newly generated SNN as `spiking_model`. The `ReLU`s have been converted to `SpikingLayers`. In addition, we have also added a spiking layer at the end. This is for compatibitlity with the chip later on, since the chips can only produce spikes and not  activations.

In [3]:
sinabs_model = from_model(ann, add_spiking_output=True)
print(sinabs_model.spiking_model)

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (1): SpikingLayer()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(20, 32, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (4): SpikingLayer()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (7): SpikingLayer()
  (8): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=128, out_features=500, bias=False)
  (11): SpikingLayer()
  (12): Linear(in_features=500, out_features=10, bias=False)
  (Spiking output): SpikingLayer()
)


## DYNAP-CNN compatible network

The next step is to convert the SNN to a `DynapcnnCompatibleNetwork`. This way we can be sure that all functionalities of our network are supported by the hardware and we can simulate the expected hardware output for testing purposes. This object will also generate the configuration objects to set up the chip.

We need to tell the chip the dimensions of the input data. This can be done either by specifying an `input_shape` argument in the constructor or including a SINABS `InputLayer` at the beginning of the model.

The class will convert the parameters (weights, biases, and thresholds) to discrete values that are supported by DYNAP-CNN. For testing purposes this can be disabled by setting `discretize` to `False`.

We can use the `dvs_input` flag to determine whether the chip should process data coming from the on-chip dynamic vision sensor (DVS).

In [4]:
# - Input dimensions
input_shape = (1, 28, 28)

# - DYNAP-CNN compatible network
dynapcnn_net = DynapcnnCompatibleNetwork(
    sinabs_model.spiking_model,
    input_shape=input_shape,
    discretize=True,
    dvs_input=False,
)
print(dynapcnn_net)

DynapcnnCompatibleNetwork(
  (sequence): Sequential(
    (0): DynapcnnLayer(
      (_conv_layer): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), bias=False)
      (_spk_layer): SpikingLayer()
      (_pool_layer): SumPool2d(norm_type=1, kernel_size=2, stride=2, ceil_mode=False)
    )
    (1): DynapcnnLayer(
      (_conv_layer): Conv2d(20, 32, kernel_size=(5, 5), stride=(1, 1), bias=False)
      (_spk_layer): SpikingLayer()
      (_pool_layer): SumPool2d(norm_type=1, kernel_size=2, stride=2, ceil_mode=False)
    )
    (2): DynapcnnLayer(
      (_conv_layer): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (_spk_layer): SpikingLayer()
      (_pool_layer): SumPool2d(norm_type=1, kernel_size=2, stride=2, ceil_mode=False)
    )
    (3): DynapcnnLayer(
      (_conv_layer): Conv2d(128, 500, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (_spk_layer): SpikingLayer()
      (_pool_layer): SumPool2d(norm_type=1, kernel_size=1, stride=1, ceil_mode=False)
    )
    (4): 

The resulting model consists of 5 `DynapcnnLayer` objects, each containing a convolutional, a spiking, and possibly a pooling layer. Because there were 5 parameter layers (as we pointed out earlier), each gets mapped into a separate layer.

## Model evaluation

Lets pass some data and see how this converted spiking model performs.

In [5]:
# Define custom dataset for spiking input data
class MNIST_Dataset(datasets.MNIST):

    def __init__(self, root, train = True, spiking=False, tWindow=100):
        super().__init__(root, train=train, download=True)
        self.spiking=spiking
        self.tWindow = tWindow


    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]

        if self.spiking:
            img = (np.random.rand(self.tWindow, 1, *img.size()) < img.numpy()/255.0).astype(float)
            img = torch.from_numpy(img).float()
        else:
            # Convert image to tensor
            img = torch.from_numpy(img.numpy()).float()
            img.unsqueeze_(0)

        return img, target

In [6]:
# Define dataloader
tWindow = 200 # ms (or) time steps

# Define test dataset loader
test_dataset = MNIST_Dataset("./data", train=False, spiking=True, tWindow=tWindow)


In [ ]:
with torch.no_grad():
    correct = 0
    samples = 0
    pbar = tqdm(test_dataset)
    for data, label in pbar:
        out = dynapcnn_net(data)
        
        # Calculate total number of spikes out
        pred = out.squeeze().sum(0)
        
        # Check if the prediction matches the label
        if pred.argmax() == label:
            correct += 1
        samples += 1
        pbar.set_postfix(acc=100*correct/samples)
        

The final accuracy of this model can now be evaluated.

In [ ]:
f"Accuracy of the  dynapcnn_net is: {100*correct/samples}%"

## Porting model to DYNAP-CNN DevKit

Similar to porting a model to `cpu` or `gpu` in pytorch, the `DynapcnnCompatibleNetwork` is a special class that supports porting a model to hardware based on DYNAP-CNN technology.

In [ ]:
# Apply model to device
dynapcnn_net.to(
    device="dynapcnndevkit:0",
    chip_layers_ordering='auto',
    monitor_layers=[8],
    config_modifier=config_modifier,
)


## Inspecting memory

We can now inspect the memory required by this model when mapped onto the chip.

> Note that this memory is not the same as the total number of kernel parameters. See the `memory_summary` documentation for more details on this.

In [ ]:
dynapcnn_net.memory_summary()

In [ ]:
# - Run network on random data
input_data = rand((1, *input_shape)) * 1000
output_data = dynapcnn_net(input_data)

# - Model is quantized, so the output will have an integer value.
print(output_data)

## DYNAP-CNN configuration

We can now extract a dynapcnn configuration object from the `DynapcnnCompatibleNetwork`, which can then be used to configure the hardware. In order to map layers of the sequential model to specific layers on the chip we can provide a list of layer indices in the order of the data flow.

In [ ]:
config = dynapcnn_net.make_config(dynapcnn_layers_ordering=[4, 2, 1, 7])

## Validation and upload to DYNAP-CNN

Finally, we only need to upload the configuration to the hardware. Before that however, it should be made sure that the way the layers are arranged is compatible.

The functionalities for validation and uploading will soon be added and explained here.